<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Agent Personas Decisions on OpenAI Assistants 

Empower multi-agent systems where specialists are defined to execute requests they were designed to fulfill.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [6]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

### Step 2 - Creating an Agent Persona

Personas leverage the Chat Completion service from the Semantic Kernel Agents package. Personas are basically prompt instructions on how the GenAI LLM should behave, respond, information and the tone it should respond in. In an Agent Chat a persona provides the behavior of how the agent should interact with other agents or assistants.

In this example, the agent will take on the persona of Dwight Eisenhower. In addition for being famous as a leader of allied forces in WW2 and a US president, Dwight Eisenhower is famous of the "Eisenhower Matrix" decision framework.  

<img width ="600px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/EisenhowerDecisonMatrix.png">

> "In preparing for battle, I have always found that plans are useless but planning is indispensable."
>
> -- <cite>Dwight Eisenhower (WW2 Allied Leader, US President)</cite> 

The Eisenhower Matrix Priorotization Framework is almost 80 years old. However, it is still actively used in current software systems. For example, Asana (SaaS project management software) uses the Eisenhower Matrix. https://asana.com/resources/eisenhower-matrix  
<img width ="600px" src="https://assets.asana.biz/transform/a4725133-d094-48e1-9e6d-f65c36f0cd59/inline-leadership-eisenhower-matrix-1-2x?io=transform:fill,width:2560&format=webp">

Execute the cell below to create a new Agent with the Dwight Eisenhower persona created by the Agent Builder. The code will not execute any instructions, it is simply "building" the Agent.  

In [7]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.13.0-alpha"  
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.13.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.13.0-alpha"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Experimental.Agents;

// Notice the agent is created with a name, instructions, and description
var name = "Dwight Eisenhower";
var instructions = """
You are a decision intelligence strategist with the knowledge of Dwight Eisenhower's Decision Prioritization Matrix. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var description = "A chat bot that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.";

#pragma warning disable SKEXP0101
var agent = await new AgentBuilder()
    .WithAzureOpenAIChatCompletion(
        model: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .WithInstructions(instructions)
    .WithName(name)
    .WithDescription(description)
    .BuildAsync();
#pragma warning restore SKEXP0101

Installed Packages Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Agents.Core, 1.13.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.13.0-alpha Microsoft.SemanticKernel.Experimental.Agents, 1.13.0-alpha

In [8]:
#pragma warning disable SKEXP0101
var thread = await agent.NewThreadAsync();

// Set of messages to ask the agent (Dwight Eisenhower persona) for advice
string[] messages = [
    "How should I priotize my typical lazy day?",
    "How does your favorite decision framework compare to other decision frameworks?"
];

foreach (var response in messages.Select(m => thread.InvokeAsync(agent, m)))
{
    await foreach (var message in response)
    {
        string speaker = message.Role == "user" ? "You" : "Dwight Eisenhower";
        Console.WriteLine($"# {speaker}: {message.Content}");
        Console.WriteLine("-------------------------------------------------");
        Console.WriteLine("-------------------------------------------------");
    }
}

// Delete the assistants and threads
await thread.DeleteAsync();
await agent.DeleteAsync();
#pragma warning restore SKEXP0101

# You: How should I priotize my typical lazy day?
-------------------------------------------------
-------------------------------------------------
# Dwight Eisenhower: When prioritizing tasks for a typical day off, you can use the Eisenhower Decision Matrix to help you determine what activities deserve your attention and when you should schedule them. The matrix categorizes activities into four quadrants:

1. **Urgent and Important (Do)**
2. **Important, but Not Urgent (Plan)**
3. **Urgent, but Not Important (Delegate or Re-schedule)**
4. **Neither Urgent nor Important (Eliminate or Minimize)**

Here's a structured approach to help you prioritize your lazy day:

### Step 1: List Your Activities
Write down all potential activities you might consider doing on a lazy day. Some examples might include:

- Taking a leisurely walk
- Reading a book
- Doing some light exercise or yoga
- Watching TV or movies
- Catching up with friends or family
- House chores (cleaning, laundry)
- Meditating

### Step 3 - Creating a Multi-agent Persona

Single agent personas are effective constructs in Semantic Kernel, but can largely be replicated in other ways. Creating multiple personas that can interact with each other is the true power of the Semantic Kernel Agent functionality. This allows multiple chat agents to interact in various ways.

Execute the cell below to illustrate how two different personas can interact in a decision-making process moderated by a decision intelligence expert that makes the final reasoning conclusiion on the decision task at hand.

In [9]:
using System.Threading;

#pragma warning disable SKEXP0110
// Create a new chat TerminationStrategy that will terminate when the final message contains the term "final decision"
private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("final decision", StringComparison.OrdinalIgnoreCase) ?? false);
}

var semanticKernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: openAIModelId,
        apiKey: openAIAPIKey)
    .Build();

var assistantDwightEisenhower = new Microsoft.SemanticKernel.Agents.ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "Dwight",
    Description = "A chat bot that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Dwight Eisenhower's Decision Prioritization Matrix. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantBenFranklin = new Microsoft.SemanticKernel.Agents.ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "Ben",
    Description = "A chat bot that replies to the message with recommending the Ben Franklin Decision Framework.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Ben Franklin's Decision Framework. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantDecisionMaker = new Microsoft.SemanticKernel.Agents.ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DecisionMaker",
    Description = "A chat bot that approves the final decision.",
    Instructions = """
    You are a decision intelligence maker. Make the final decision and describe the reasoning from the recommendations.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea. Use the words final decision to approve the final decision.
    """
};

var chat =
    new AgentGroupChat(assistantDwightEisenhower, assistantBenFranklin, assistantDecisionMaker)
    {
        ExecutionSettings =
            new()
            {
                // Below is a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approve".
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        // Only decision maker may approve.
                        Agents = [assistantDecisionMaker],
                        // Limit total number of turns
                        MaximumIterations = 5,
                    }
            }
    };

#pragma warning disable SKEXP0001
// Invoke chat and display messages.
string input = "Decide whether to purchase a new car or repair the existing one. The new car is more expensive but has better fuel efficiency and safety features. The existing car is reliable and has sentimental value. Repairing the existing car is cheaper but may require more frequent repairs in the future. The new car is more environmentally friendly and has a better resale value. The existing car has a lower insurance cost and maintenance cost. The new car has a longer warranty and better financing options. The existing car has a lower monthly payment and is already paid off. The new car has a higher safety rating and better crash test results. The existing car has a lower carbon footprint and is more familiar to drive. The new car has a higher resale value and better technology features. The existing car has a lower total cost of ownership and is more affordable to maintain. The new car has a higher reliability rating and better performance. The existing car has a lower depreciation rate and is more cost-effective to own. The new car has a higher customer satisfaction rating and better brand reputation. The existing car has a lower environmental impact and is more sustainable to operate. The new car has a higher fuel efficiency rating and better fuel economy. The existing car has a lower carbon footprint and is more fuel-efficient to operate. The new car has a higher safety rating and better crash test results. The existing car has a lower insurance cost and maintenance cost. The new car has a longer warranty and better financing options. The existing car has a lower monthly payment and is already paid off. The new car has a higher safety rating and better crash test results. The existing car has a lower carbon footprint and is more familiar to drive. The new car has a higher resale value and better technology features. The existing car has a lower total cost of ownership and is more affordable to maintain. The new car has a higher reliability rating and better performance. The existing car has a lower depreciation rate and is more cost-effective to own. The new car has a higher customer satisfaction rating and better brand reputation. The existing car has a lower environmental impact and is more sustainable to operate. The new car has a higher fuel efficiency rating and better fuel economy. The existing car has a lower carbon footprint and is more fuel-efficient to operate. The new car has a higher safety rating and better crash test results. The existing car has a lower insurance cost and maintenance cost. The new car has a longer warranty and better financing options. The existing car has a lower monthly payment and is already paid off.";
chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

Console.WriteLine($"# IS COMPLETE: {chat.IsComplete}");



# user: 'Decide whether to purchase a new car or repair the existing one. The new car is more expensive but has better fuel efficiency and safety features. The existing car is reliable and has sentimental value. Repairing the existing car is cheaper but may require more frequent repairs in the future. The new car is more environmentally friendly and has a better resale value. The existing car has a lower insurance cost and maintenance cost. The new car has a longer warranty and better financing options. The existing car has a lower monthly payment and is already paid off. The new car has a higher safety rating and better crash test results. The existing car has a lower carbon footprint and is more familiar to drive. The new car has a higher resale value and better technology features. The existing car has a lower total cost of ownership and is more affordable to maintain. The new car has a higher reliability rating and better performance. The existing car has a lower depreciation rate 